<a href="https://colab.research.google.com/github/faraznshaikh/Awesome-Design-Tools/blob/master/COGS_118B_final_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Final project notebook:

Ideas
“We can make an algorithm that randomly samples from a bunch of datasets so our shit’s generalizable af” - Ribhu 
We can use tSNE to project n-facial features onto a 2-d graph and use some clustering algorithm to classify facial expressions

Description: We are going to build a web application called Phace where the backend uses unsupervised learning to analyze and extrapolate information from facial features. The way we plan to implement this is to have a front end form which takes in a picture of human face which will be transported to our backend algorithm which will analyze it and give us information about certain characteristics of the face such as determining facial expressions such as smiles and frowns. We plan to implement the front end using React and Bootstrap and the backend using Node and Python.

Pipeline 1:
Image -> Dim Red (PCA or tSNE) -> Clustering -> Convolutional Neural Network (trained on cluster labels)

Pipeline 2:
Image -> Deep Autoencoder (Encoder CNN and Decoder CNN)

Idea is to test out both the methods and compare them, then use the best method in the webapp.


Datasets:

*   US Faces: http://wilmabainbridge.com/facememorability2.html 
* http://www2.ece.ohio-state.edu/~aleix/ARdatabase.html



Resources:
* https://www.sciencedirect.com/science/article/abs/pii/S1005888513600382 - Hybrid PCA and LBP for extracting facial features
* https://www.cl.cam.ac.uk/~pr10/publications/ssci15.pdf - Feature reduction
* http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.167.9632&rep=rep1&type=pdf - Improving dim red
* https://arxiv.org/ftp/arxiv/papers/1505/1505.04058.pdf - Facial expression classification in real time
* https://www.ncbi.nlm.nih.gov/pubmed/27875237 - Autoencoders
* https://github.com/ageitgey/face_recognition - Cluster code



In [0]:
import numpy as np
import pandas as pd
import sklearn
import PIL 
import cv2
import glob 
#import imageio

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [0]:
!ls /content/gdrive/Team\ Drives/Cogs\ 118B\ Group\ Project/

'118b Video.gslides'  'Link to Notebook.gdoc'   SadLandmarks
 cohn-kanade	       misc		        train
 colorferet.tar        MiscLandmarks	       'trainNew (1)'
 dataset	      'Project Planning.gdoc'   valid
 happy		      'project talk.docx'       validNew
 HappyLandmarks       'project talk.gdoc'       vgg16_weights.h5
 images		       sad


# Pre-Processing

In [0]:
from scipy import misc
png_happy = []
for image_path1 in glob.glob("/content/gdrive/Team Drives/Cogs 118B Group Project/happy/*.png"):
    png_happy.append(misc.imread(image_path1))  

im_happy = np.asarray(png_happy)

print ('Importing done...', im_happy.shape)
print(im_happy[0])


png_sad = []
for image_path2 in glob.glob("/content/gdrive/Team Drives/Cogs 118B Group Project/sad/*.png"):
    png_sad.append(misc.imread(image_path2))  

im_sad = np.asarray(png_sad)

print ('Importing done...', im_sad.shape)
print(im_sad[0])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  after removing the cwd from sys.path.


Importing done... (114, 490, 640)
[[ 55 104 154 ...   0   0   0]
 [ 60 106 151 ...   0   0   0]
 [  0   0   0 ...   0   0   0]
 ...
 [ 39  42  42 ... 200 203 201]
 [ 40  41  40 ... 195 193 194]
 [ 39  42  42 ... 200 203 201]]


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  


Importing done... (99, 490, 640)
[[ 48 105 143 ...  78  43  21]
 [ 57 110 148 ...  67  36  16]
 [  0   0   0 ...   0   0   0]
 ...
 [  9   8   7 ...  12  12  12]
 [  5   6  11 ...  13  13  11]
 [  9   8   7 ...  12  12  12]]


In [0]:
print(im_sad[0])

[[ 48 105 143 ...  78  43  21]
 [ 57 110 148 ...  67  36  16]
 [  0   0   0 ...   0   0   0]
 ...
 [  9   8   7 ...  12  12  12]
 [  5   6  11 ...  13  13  11]
 [  9   8   7 ...  12  12  12]]


# Dimensionality Reduction

In [0]:
from sklearn.decomposition import PCA
from numpy import linalg as LA

#perform PCA on each image in image_array and store in pca_array
#pca = PCA(n_components=300)
#pca_array = []
#final_array = []

#add all images together to find mean image
for img_i in range(len(im_happy)):
    if img_i == 0:
      pca_array = im_happy[img_i]
    else:
      pca_array = np.add(pca_array, im_happy[img_i])
    mean_array = np.array(pca_array)
    mean_array = mean_array/len(im_happy)
    
#find eigvecs and eigvals
Z = np.tile(mean_array,len(im_happy))
vec,val = LA.eig(np.matmul(np.transpose(Z),Z))
    
#eigsort function for sorting in decreasing order
def eigsort(V, D):
  eigvals = np.diag(D)
  lohival = np.sort(eigvals, axis=0)
  lohiindex = np.argsort(eigvals, axis=0)
  eigvec = np.flipud(lohival)
  index = np.flipud(lohiindex)
  Dsort = np.diag(eigvec)
  M=len(eigvec)
  Vsort = np.zeros((M,M))
  for i in range(M):
    Vsort[:,i] = V[:,index[i]]
  return (Vsort, Dsort)

#sort eigvecs and eigvals
vec,val = eigsort(vec,val)

#find principal components
U = np.matmul(Z, vec)

#normalize vectors in U
for img_i in range(len(U)):
  U[img_i] = LA.norm(U[img_i])

#U now contains the principal components

In [0]:
import scipy.misc
n = 0
for item1 in im_happy:
  n= n +1 
  outfile_happy = '%s.jpg' % (n)
  scipy.misc.imsave(/outfile_happy,item2)
  
m = 0
for item2 in im_sad:
  m = m +1 
  outfile_sad = '%s.jpg' % (m)
  scipy.misc.imsave(outfile_sad,item)
  
  
  
#scipy.misc.imsave('outfile0.jpg',im[0])
#scipy.misc.imsave('outfile1.jpg', pca_array[0])

In [0]:
picture_arrayHappy =[]
for pictures1 in glob.glob("*.jpg"):
  picture_arrayHappy.append(misc.imread(pictures1))
  
picture_arraySad =[]
for pictures2 in glob.glob("*.jpg"):
  picture_arraySad.append(misc.imread(pictures2))

In [0]:
print(len(picture_arraySad))

# Clustering

In [0]:
#TODO
! pip install CMake
! pip install Dlib


In [0]:
!pip install face_recognition

In [0]:
from PIL import Image, ImageDraw
import face_recognition

landmarks_array_Happy = []
index_H = 0 
for pictures in glob.glob("/content/gdrive/Team Drives/Cogs 118B Group Project/happy/*.png"):
      # Load the jpg file into a numpy array
      image = face_recognition.load_image_file(pictures)

      # Find all facial features in all the faces in the image
      face_landmarks_list = face_recognition.face_landmarks(image)

      #print("I found {} face(s) in this photograph.".format(len(face_landmarks_list)))

      # Create a PIL imagedraw object so we can draw on the picture
      pil_image = Image.fromarray(image)
      d = ImageDraw.Draw(pil_image)

      for face_landmarks in face_landmarks_list:

          # Print the location of each facial feature in this image
          #for facial_feature in face_landmarks.keys():
              #print("The {} in this face has the following points: {}".format(facial_feature, face_landmarks[facial_feature]))

          # Let's trace out each facial feature in the image with a line!
          for facial_feature in face_landmarks.keys():
              d.line(face_landmarks[facial_feature], width=5)

      # Show the picture
      pil_image.show()
      
      index_H = index_H+1
      scipy.misc.imsave('/content/gdrive/Team Drives/Cogs 118B Group Project/HappyLandmarks/'+ '%s.png' %(index_H),pil_image)
      
      
      landmarks_array_Happy.append(pil_image)

In [0]:
print(len(landmarks_array_Happy))

In [0]:
landmarks_array_Sad = []
index_H = 0 
for pictures in glob.glob("/content/gdrive/Team Drives/Cogs 118B Group Project/sad/*.png"):
      # Load the jpg file into a numpy array
      image = face_recognition.load_image_file(pictures)

      # Find all facial features in all the faces in the image
      face_landmarks_list = face_recognition.face_landmarks(image)

      #print("I found {} face(s) in this photograph.".format(len(face_landmarks_list)))

      # Create a PIL imagedraw object so we can draw on the picture
      pil_image = Image.fromarray(image)
      d = ImageDraw.Draw(pil_image)

      for face_landmarks in face_landmarks_list:

          # Print the location of each facial feature in this image
          #for facial_feature in face_landmarks.keys():
              #print("The {} in this face has the following points: {}".format(facial_feature, face_landmarks[facial_feature]))

          # Let's trace out each facial feature in the image with a line!
          for facial_feature in face_landmarks.keys():
              d.line(face_landmarks[facial_feature], width=5)

      # Show the picture
      pil_image.show()
      
      index_H = index_H+1
      scipy.misc.imsave('/content/gdrive/Team Drives/Cogs 118B Group Project/SadLandmarks/'+ '%s.png' %(index_H),pil_image)
      
      
      landmarks_array_Sad.append(pil_image)

In [0]:
landmarks_array_Misc = []
index_H = 0 
for pictures in glob.glob("/content/gdrive/Team Drives/Cogs 118B Group Project/misc/*.png"):
      # Load the jpg file into a numpy array
      image = face_recognition.load_image_file(pictures)

      # Find all facial features in all the faces in the image
      face_landmarks_list = face_recognition.face_landmarks(image)

      #print("I found {} face(s) in this photograph.".format(len(face_landmarks_list)))

      # Create a PIL imagedraw object so we can draw on the picture
      pil_image = Image.fromarray(image)
      d = ImageDraw.Draw(pil_image)

      for face_landmarks in face_landmarks_list:

          # Print the location of each facial feature in this image
          #for facial_feature in face_landmarks.keys():
              #print("The {} in this face has the following points: {}".format(facial_feature, face_landmarks[facial_feature]))

          # Let's trace out each facial feature in the image with a line!
          for facial_feature in face_landmarks.keys():
              d.line(face_landmarks[facial_feature], width=5)

      # Show the picture
      pil_image.show()
      
      index_H = index_H+1
      scipy.misc.imsave('/content/gdrive/Team Drives/Cogs 118B Group Project/MiscLandmarks/'+ '%s.png' %(index_H),pil_image)
      
      
      landmarks_array_Misc.append(pil_image)

In [0]:
scipy.misc.imsave('landmarksFace.jpg',pil_image)

# Convolutional Neural Network

In [0]:
# CNN from scratch - Baseline model
# Model from F_Chollet's script:
# https://gist.github.com/fchollet/0830affa1f7f19fd47b06d4cf89ed44d
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K


# dimensions of our images.
img_width, img_height = 640, 490

train_data_dir = '/content/gdrive/Team Drives/Cogs 118B Group Project/trainNew'
validation_data_dir = '/content/gdrive/Team Drives/Cogs 118B Group Project/validNew'
nb_train_samples = 467
nb_validation_samples = 114
epochs = 50
batch_size = 16

if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='sparse')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='sparse')

model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
validation_steps=nb_validation_samples // batch_size)

Found 462 images belonging to 3 classes.
Found 114 images belonging to 3 classes.
Epoch 1/50
29/29 [==============================] - 302s 10s/step - loss: 0.3592 - acc: 0.6081 - val_loss: 0.2847 - val_acc: 0.6429
Epoch 2/50
29/29 [==============================] - 38s 1s/step - loss: 0.4266 - acc: 0.6272 - val_loss: 0.8134 - val_acc: 0.6429
Epoch 3/50
29/29 [==============================] - 39s 1s/step - loss: 0.4313 - acc: 0.6275 - val_loss: 0.1627 - val_acc: 0.6633
Epoch 4/50
29/29 [==============================] - 39s 1s/step - loss: 0.4313 - acc: 0.6269 - val_loss: 0.8134 - val_acc: 0.6020
Epoch 5/50
29/29 [==============================] - 39s 1s/step - loss: 0.4171 - acc: 0.6278 - val_loss: 0.1627 - val_acc: 0.6020
Epoch 6/50
29/29 [==============================] - 39s 1s/step - loss: 0.4219 - acc: 0.6281 - val_loss: 0.4880 - val_acc: 0.7041
Epoch 7/50
29/29 [==============================] - 39s 1s/step - loss: 0.4029 - acc: 0.6281 - val_loss: 4.8657e-08 - val_acc: 0.6122
Ep

In [0]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
from keras import applications


# dimensions of our images.
img_width, img_height = 640, 490

train_data_dir = '/content/gdrive/Team Drives/Cogs 118B Group Project/trainNew'
validation_data_dir = '/content/gdrive/Team Drives/Cogs 118B Group Project/validNew'
top_model_weights_path = 'bottleneck_fc_model.h5'
nb_train_samples = 467
nb_validation_samples = 114
epochs = 50
batch_size = 16

bft = []
bfv = []

def save_bottlebeck_features():
    datagen = ImageDataGenerator(rescale=1. / 255)

    # build the VGG16 network
    model = applications.VGG16(include_top=False, weights='imagenet')

    generator = datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
    bottleneck_features_train = model.predict_generator(
        generator, nb_train_samples // batch_size)
    np.save(open('bottleneck_features_train.npy', 'wb'),
            bottleneck_features_train)

    generator = datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
    bottleneck_features_validation = model.predict_generator(
        generator, nb_validation_samples // batch_size)
    np.save(open('bottleneck_features_validation.npy', 'wb'),
            bottleneck_features_validation)


def train_top_model():
    train_data = np.load(open('bottleneck_features_train.npy','rb'))
    train_labels = np.array([0] * (nb_train_samples // 2) + [1] * (nb_train_samples // 2))
    train_labels = train_labels[:len(train_labels)-2]

    validation_data = np.load(open('bottleneck_features_validation.npy','rb'))
    print(len(validation_data))
    validation_labels = np.array([0] * (nb_validation_samples // 2) + [1] * (nb_validation_samples // 2))
    validation_labels = validation_labels[:len(validation_labels)-2]
    print(len(validation_labels))

    model = Sequential()
    model.add(Flatten(input_shape=train_data.shape[1:]))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer='rmsprop',
                  loss='binary_crossentropy', metrics=['accuracy'])

    result = model.fit(train_data, train_labels,
              epochs=epochs,
              batch_size=batch_size,
              validation_data=(validation_data, validation_labels))
    model.save_weights(top_model_weights_path)


save_bottlebeck_features()
train_top_model()

Found 466 images belonging to 3 classes.
Found 114 images belonging to 3 classes.
112
112
Train on 464 samples, validate on 112 samples
Epoch 1/50
464/464 [==============================] - 2s 5ms/step - loss: 7.8179 - acc: 0.4957 - val_loss: 7.9151 - val_acc: 0.5089
Epoch 2/50
464/464 [==============================] - 1s 3ms/step - loss: 8.0243 - acc: 0.5022 - val_loss: 7.9151 - val_acc: 0.5089
Epoch 3/50
464/464 [==============================] - 1s 3ms/step - loss: 8.0243 - acc: 0.5022 - val_loss: 7.9151 - val_acc: 0.5089
Epoch 4/50
464/464 [==============================] - 1s 3ms/step - loss: 8.0243 - acc: 0.5022 - val_loss: 7.9151 - val_acc: 0.5089
Epoch 5/50
464/464 [==============================] - 1s 3ms/step - loss: 8.0243 - acc: 0.5022 - val_loss: 7.9151 - val_acc: 0.5089
Epoch 6/50
464/464 [==============================] - 1s 3ms/step - loss: 8.0243 - acc: 0.5022 - val_loss: 7.9151 - val_acc: 0.5089
Epoch 7/50
464/464 [==============================] - 1s 3ms/step - loss

In [0]:
# Using pre-trained VGG16 weights
# Model from F_Chollet's script:
# https://gist.github.com/fchollet/7eb39b44eb9e16e59632d25fb3119975

from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
from keras import backend as K
from keras.models import Model

# path to the model weights files.
weights_path = '/content/gdrive/Team Drives/Cogs 118B Group Project/vgg16_weights.h5'
top_model_weights_path = 'bottleneck_fc_model.h5'

# dimensions of our images.
img_width, img_height = 640, 490

train_data_dir = '/content/gdrive/Team Drives/Cogs 118B Group Project/trainNew'
validation_data_dir = '/content/gdrive/Team Drives/Cogs 118B Group Project/validNew'
nb_train_samples = 467
nb_validation_samples = 114
epochs = 50
batch_size = 16

# build the VGG16 network
model = applications.VGG16(weights='imagenet', include_top=False, input_shape = (img_height,img_width,3))
print('Model loaded.')

fresh_m = Sequential()
for layer in model.layers:
  fresh_m.add(layer)
  
# build a classifier model to put on top of the convolutional model
top_model = Sequential()
top_model.add(Flatten(input_shape=model.output_shape[1:]))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(1, activation='sigmoid'))

# note that it is necessary to start with a fully-trained
# classifier, including the top classifier,
# in order to successfully do fine-tuning
top_model.load_weights(top_model_weights_path)

# add the model on top of the convolutional base
# model = Model(inputs= model.input, outputs= top_model(model.output))
fresh_m.add(top_model)

# set the first 25 layers (up to the last conv block)
# to non-trainable (weights will not be updated)
for layer in model.layers[:25]:
    layer.trainable = False

# compile the model with a SGD/momentum optimizer
# and a very slow learning rate.
fresh_m.compile(loss='binary_crossentropy',
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

# prepare data augmentation configuration
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary')

# fine-tune the model
result = fresh_m.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
nb_val_samples=nb_validation_samples)

Model loaded.
Found 466 images belonging to 3 classes.
Found 114 images belonging to 3 classes.


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:86: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:86: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras_pre..., steps_per_epoch=29, epochs=50, validation_data=<keras_pre..., validation_steps=114)`


Epoch 1/50
29/29 [==============================] - 144s 5s/step - loss: 15.7360 - acc: 0.1961 - val_loss: 15.6825 - val_acc: 0.1935
Epoch 2/50
29/29 [==============================] - 128s 4s/step - loss: 15.4310 - acc: 0.2088 - val_loss: 15.7419 - val_acc: 0.1916
Epoch 3/50
29/29 [==============================] - 128s 4s/step - loss: 15.2571 - acc: 0.2152 - val_loss: 15.6726 - val_acc: 0.1935
Epoch 4/50
29/29 [==============================] - 127s 4s/step - loss: 15.8051 - acc: 0.1899 - val_loss: 15.6787 - val_acc: 0.1933
Epoch 5/50
29/29 [==============================] - 128s 4s/step - loss: 16.1097 - acc: 0.1855 - val_loss: 15.6924 - val_acc: 0.1923
Epoch 6/50
29/29 [==============================] - 128s 4s/step - loss: 15.5269 - acc: 0.2006 - val_loss: 15.7320 - val_acc: 0.1904
Epoch 7/50
29/29 [==============================] - 128s 4s/step - loss: 15.8747 - acc: 0.1791 - val_loss: 15.6627 - val_acc: 0.1966
Epoch 8/50
29/29 [==============================] - 127s 4s/step - lo

KeyboardInterrupt: ignored

# Evaluation

In [0]:
import matplotlib.pyplot as plt

train_loss = result.history['loss']
val_loss   = result.history['val_loss']
train_acc  = result.history['acc']
val_acc    = result.history['val_acc']
xc         = range(epochs)

plt.figure()
plt.plot(xc, train_loss)
plt.plot(xc, val_loss)
plt.legend(('training','validation'))
plt.xlabel('Number of Epochs')
plt.ylabel('Loss')
plt.show()

In [0]:
plt.figure()
plt.plot(xc, train_acc)
plt.plot(xc, val_acc)
plt.legend(('training','validation'))
plt.xlabel('Number of Epochs')
plt.ylabel('Accuracy')
plt.show()